In [10]:
import cv2
import concurrent.futures
import os
import time
from skimage.feature import local_binary_pattern
import shutil

In [11]:
classes = ['COVID', 'Lung_Opacity', 'Normal', 'Viral_Pneumonia']
root_dir = 'COVID-19_Radiography_Dataset'
train_dirs = {}
val_dirs = {}

In [12]:
lbp_dir = os.path.join(root_dir, 'lbp')
os.makedirs(lbp_dir, exist_ok=True)

for category in classes: 
    os.makedirs(os.path.join(lbp_dir, category), exist_ok=True)

In [13]:
train_dir = os.path.join(lbp_dir, 'train')
val_dir = os.path.join(lbp_dir, 'val')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

for index, folder in enumerate([train_dir, val_dir]):
    for category in classes:
        category_dir = os.path.join(folder, category)
        os.makedirs(category_dir, exist_ok=True)
        if index < 1:
            train_dirs[category] = category_dir
        else:
            val_dirs[category] = category_dir

In [14]:
def apply_lbp(image, points=8, radius=1, method='default'):
  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  lbp_image = local_binary_pattern(gray_image, points, radius, method=method)
  return lbp_image


def filter_images(classname):
  img_dir = os.path.join(root_dir, classname, 'images')
  start_time = time.time()
  for image_name in os.listdir(img_dir):
    image = cv2.imread(os.path.join(img_dir, image_name))
    filtered_image = apply_lbp(image, method='default')
    cv2.imwrite(os.path.join(lbp_dir, classname, image_name), filtered_image)
  end_time = time.time()
  print(f'Done with {classname} original images ({(end_time - start_time):.6f}s)')

In [15]:
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
  # Use starmap to unpack arguments for filter_image function
  result = executor.map(filter_images, classes)

Done with Viral_Pneumonia original images (62.980737s)
Done with COVID original images (131.896770s)
Done with Lung_Opacity original images (202.420816s)
Done with Normal original images (309.140007s)


In [16]:
def move_data(category):
    category_path = os.path.join(root_dir, 'lbp', category)
    images = os.listdir(category_path)

    # Ensure the category directories exist in train and val directories
    train_category_dir = train_dirs[category]
    val_category_dir = val_dirs[category]
    os.makedirs(train_category_dir, exist_ok=True)
    os.makedirs(val_category_dir, exist_ok=True)

    train_images = images[:int(len(images) * 0.8)]
    val_images = images[int(len(images) * 0.8):]

    # Copy train images
    start_time = time.time()
    for img in train_images:
        src_path = os.path.join(category_path, img)
        dst_path = os.path.join(train_category_dir, img)
        if not os.path.exists(dst_path):
            shutil.move(src_path, dst_path)
    end_time = time.time()
    print(f'Done with {category} train data ({(end_time - start_time):.6f}s)')

    # Copy val images
    start_time = time.time()
    for img in val_images:
        src_path = os.path.join(category_path, img)
        dst_path = os.path.join(val_category_dir, img)
        if not os.path.exists(dst_path):
            shutil.move(src_path, dst_path)
    end_time = time.time()
    print(f'Done with {category} val data ({(end_time - start_time):.6f}s)')

In [17]:
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
  # Use starmap to unpack arguments for filter_image function
  result = executor.map(move_data, classes)

Done with Viral_Pneumonia train data (1.937483s)
Done with Viral_Pneumonia val data (0.445350s)
Done with COVID train data (5.225930s)
Done with COVID val data (1.197302s)
Done with Lung_Opacity train data (8.880729s)
Done with Lung_Opacity val data (2.102913s)
Done with Normal train data (14.258892s)
Done with Normal val data (2.742465s)
